In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/movies.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [3]:
users = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/users.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [4]:
ratings = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/ratings.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [5]:
training_set = np.array(pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-100k/u1.base' , delimiter='\t') , dtype='int')
testing_set = np.array(pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-100k/u1.test' , delimiter='\t') , dtype='int')

In [6]:
total_users = int(max(max(training_set[:,0]),max(testing_set[:,0])))
total_movies = int(max(max(training_set[:,1]),max(testing_set[:,1])))

In [7]:
def convert_data(data):
    ret_data = []
    for i in range(1,total_users+1):
        ratings = np.zeros(total_movies)
        for j in range(0,data.shape[0]):
            if data[j][0] == i:
                ratings[data[j][1] - 1] = data[j][2] 
        ret_data.append(list(ratings))
    return ret_data

In [8]:
training_set_new = convert_data(training_set)

In [9]:
testing_set_new = convert_data(testing_set)

In [10]:
xtrain = torch.FloatTensor(training_set_new)
xtest = torch.FloatTensor(testing_set_new)

In [17]:
class SAE(nn.Module):
    def __init__(self , ):
        super(SAE,self).__init__()
        self.fc1 = nn.Linear(total_movies , 20)
        self.fc2 = nn.Linear(20 , 10)
        self.fc3 = nn.Linear(10 , 20)
        self.fc4 = nn.Linear(20 , total_movies)
        self.activation = nn.Sigmoid()
        
    def forward_propagation(self , x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
    
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters() , lr = 0.01 ,weight_decay = 0.5)

In [22]:
nb_epoch = 200
for epoch in range(1 , nb_epoch + 1):
    train_loss = 0
    ctr = 0.
    for user in range(total_users):
        inp_ = Variable(xtrain[user]).unsqueeze(0)
        target = inp_.clone()
        if torch.sum(target.data > 0) > 0:
            out_ = sae.forward_propagation(inp_)
            target.require_grad = False
            out_[target == 0] = 0
            loss = criterion(out_ , target)
            mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt((loss.data[0])*(mean_corrector))
            ctr += 1.
            optimizer.step()
    print('epoch : ' , epoch , 'loss : ' , str(train_loss/ctr))

/home/nilesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


epoch :  1 loss :  tensor(1.7537)
epoch :  2 loss :  tensor(1.0962)
epoch :  3 loss :  tensor(1.0531)
epoch :  4 loss :  tensor(1.0384)
epoch :  5 loss :  tensor(1.0309)
epoch :  6 loss :  tensor(1.0266)
epoch :  7 loss :  tensor(1.0237)
epoch :  8 loss :  tensor(1.0220)
epoch :  9 loss :  tensor(1.0207)
epoch :  10 loss :  tensor(1.0196)
epoch :  11 loss :  tensor(1.0187)
epoch :  12 loss :  tensor(1.0182)
epoch :  13 loss :  tensor(1.0177)
epoch :  14 loss :  tensor(1.0174)
epoch :  15 loss :  tensor(1.0173)
epoch :  16 loss :  tensor(1.0168)
epoch :  17 loss :  tensor(1.0167)
epoch :  18 loss :  tensor(1.0164)
epoch :  19 loss :  tensor(1.0164)
epoch :  20 loss :  tensor(1.0162)
epoch :  21 loss :  tensor(1.0160)
epoch :  22 loss :  tensor(1.0160)
epoch :  23 loss :  tensor(1.0156)
epoch :  24 loss :  tensor(1.0156)
epoch :  25 loss :  tensor(1.0155)
epoch :  26 loss :  tensor(1.0156)
epoch :  27 loss :  tensor(1.0154)
epoch :  28 loss :  tensor(1.0152)
epoch :  29 loss :  tensor(1.

In [24]:
test_loss = 0
ctr = 0.
for user in range(total_users):
    inp_ = Variable(xtrain[user]).unsqueeze(0)
    target = Variable(xtest[user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        out_ = sae.forward_propagation(inp_)
        target.require_grad = False
        out_[target == 0] = 0
        loss = criterion(out_ , target)
        mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt((loss.data[0])*(mean_corrector))
        ctr += 1.
print('loss : ' , str(test_loss/ctr))

/home/nilesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':


loss :  tensor(0.9512)
